In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cardiovascular-diseases-risk-prediction-dataset/CVD_cleaned.csv


In [ ]:
import pandas as pd
from sklearn import preprocessing
import numpy as np

from sklearn import metrics
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve,auc
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

import keras
from sklearn.model_selection import train_test_split

In [60]:
df = pd.read_csv('/kaggle/input/cardiovascular-diseases-risk-prediction-dataset/CVD_cleaned.csv')

In [61]:
df.head()

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,70-74,150.0,32.66,14.54,Yes,0.0,30.0,16.0,12.0
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,165.0,77.11,28.29,No,0.0,30.0,0.0,4.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,163.0,88.45,33.47,No,4.0,12.0,3.0,16.0
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,75-79,180.0,93.44,28.73,No,0.0,30.0,30.0,8.0
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,191.0,88.45,24.37,Yes,0.0,8.0,4.0,0.0


# **No null values**

In [7]:
df.isnull().sum()

General_Health                  0
Checkup                         0
Exercise                        0
Heart_Disease                   0
Skin_Cancer                     0
Other_Cancer                    0
Depression                      0
Diabetes                        0
Arthritis                       0
Sex                             0
Age_Category                    0
Height_(cm)                     0
Weight_(kg)                     0
BMI                             0
Smoking_History                 0
Alcohol_Consumption             0
Fruit_Consumption               0
Green_Vegetables_Consumption    0
FriedPotato_Consumption         0
dtype: int64

# **Numerical columns**

In [8]:
num = df.select_dtypes('number').columns
num_tot = len(num)
print(num)
print(num_tot)

Index(['Height_(cm)', 'Weight_(kg)', 'BMI', 'Alcohol_Consumption',
       'Fruit_Consumption', 'Green_Vegetables_Consumption',
       'FriedPotato_Consumption'],
      dtype='object')
7


**There are 7 numerical columns**

In [9]:
df_num = df[['Height_(cm)', 'Weight_(kg)', 'BMI','Alcohol_Consumption', 'Fruit_Consumption','Green_Vegetables_Consumption', 'FriedPotato_Consumption']]

In [10]:
df_num.head()

,Height_(cm),Weight_(kg),BMI,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,150.0,32.66,14.54,0.0,30.0,16.0,12.0
1,165.0,77.11,28.29,0.0,30.0,0.0,4.0
2,163.0,88.45,33.47,4.0,12.0,3.0,16.0
3,180.0,93.44,28.73,0.0,30.0,30.0,8.0
4,191.0,88.45,24.37,0.0,8.0,4.0,0.0


# **Preprocessing numericals**

In [11]:
scaler = preprocessing.StandardScaler()
names = df_num.columns
d = scaler.fit_transform(df_num)
scaled_df_num = pd.DataFrame(d, columns=names)
scaled_df_num.head()

,Height_(cm),Weight_(kg),BMI,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,-1.934250,-2.386180,-2.159696,-0.621527,0.006625,0.059597,0.664502
1,-0.526857,-0.303547,-0.051548,-0.621527,0.006625,-1.012342,-0.267579
2,-0.714510,0.227770,0.742649,-0.133707,-0.716973,-0.811354,1.130543
3,0.880535,0.461569,0.015913,-0.621527,0.006625,0.997544,0.198462
4,1.912623,0.227770,-0.652562,-0.621527,-0.877772,-0.744358,-0.733620


# **Categorical columns**

In [12]:
obj = df.select_dtypes('object').columns
obj_tot = len(obj)
print(obj)
print(obj_tot)

Index(['General_Health', 'Checkup', 'Exercise', 'Heart_Disease', 'Skin_Cancer',
       'Other_Cancer', 'Depression', 'Diabetes', 'Arthritis', 'Sex',
       'Age_Category', 'Smoking_History'],
      dtype='object')
12


In [13]:
# all columns -> numerical + object = 19
num_tot + obj_tot

19

In [62]:
df_obj = df[['General_Health', 'Checkup', 'Exercise', 'Skin_Cancer','Other_Cancer', 'Depression', 'Diabetes', 'Arthritis', 'Sex','Age_Category', 'Smoking_History']]

In [63]:
df_obj.columns

Index(['General_Health', 'Checkup', 'Exercise', 'Skin_Cancer', 'Other_Cancer',
       'Depression', 'Diabetes', 'Arthritis', 'Sex', 'Age_Category',
       'Smoking_History'],
      dtype='object')

# **Preprocessing categoricals**

**General_Health**

In [64]:
df_obj['General_Health'].value_counts()

Very Good    110395
Good          95364
Excellent     55954
Fair          35810
Poor          11331
Name: General_Health, dtype: int64

In [ ]:
df_obj['General_Health'] = df_obj['General_Health'].map({'Poor':0, 'Fair':1, 'Good':2, 'Very Good': 3, 'Excellent':4})

In [66]:
df_obj['General_Health'].value_counts()

3    110395
2     95364
4     55954
1     35810
0     11331
Name: General_Health, dtype: int64

**Checkup**

In [67]:
df_obj['Checkup'].value_counts()

Within the past year       239371
Within the past 2 years     37213
Within the past 5 years     17442
5 or more years ago         13421
Never                        1407
Name: Checkup, dtype: int64

In [ ]:
df_obj['Checkup'] = df_obj['Checkup'].map({'Never':0, '5 or more years ago':1, 'Within the past 5 years':2, 'Within the past 2 years': 3, 'Within the past year':4})

In [69]:
df_obj['Checkup'].value_counts()

4    239371
3     37213
2     17442
1     13421
0      1407
Name: Checkup, dtype: int64

**Exercise, Skin_cancer, Other_Cancer, Depression, Arthritis, Sex, Smoking_History**

In [ ]:
df_obj['Exercise'] = df_obj['Exercise'].map({'No':0,'Yes':1})
df_obj['Skin_Cancer'] = df_obj['Skin_Cancer'].map({'No':0,'Yes':1})
df_obj['Other_Cancer'] = df_obj['Other_Cancer'].map({'No':0,'Yes':1})
df_obj['Depression'] = df_obj['Depression'].map({'No':0,'Yes':1})
df_obj['Arthritis'] = df_obj['Arthritis'].map({'No':0,'Yes':1})
df_obj['Sex'] = df_obj['Sex'].map({'Male':0,'Female':1})
df_obj['Smoking_History'] = df_obj['Smoking_History'].map({'No':0,'Yes':1})

**Diabetes**

In [78]:
df_obj['Diabetes'].value_counts()

No                                            259141
Yes                                            40171
No, pre-diabetes or borderline diabetes         6896
Yes, but female told only during pregnancy      2646
Name: Diabetes, dtype: int64

In [ ]:
df_obj['Diabetes'] = df_obj['Diabetes'].map({'No':0,'No, pre-diabetes or borderline diabetes':1, 'Yes':2, 'Yes, but female told only during pregnancy':3})

In [81]:
df_obj['Diabetes'].value_counts()

0    259141
2     40171
1      6896
3      2646
Name: Diabetes, dtype: int64

**Age_Category**

In [83]:
df_obj['Age_Category'].value_counts()

65-69    33434
60-64    32418
70-74    31103
55-59    28054
50-54    25097
80+      22271
40-44    21595
45-49    20968
75-79    20705
35-39    20606
18-24    18681
30-34    18428
25-29    15494
Name: Age_Category, dtype: int64

In [87]:
df_age_cat = df_obj['Age_Category'].to_frame()
df_age_cat = pd.get_dummies(df_age_cat,columns=df_age_cat.columns)

In [88]:
df_age_cat

,Age_Category_18-24,Age_Category_25-29,Age_Category_30-34,Age_Category_35-39,Age_Category_40-44,Age_Category_45-49,Age_Category_50-54,Age_Category_55-59,Age_Category_60-64,Age_Category_65-69,Age_Category_70-74,Age_Category_75-79,Age_Category_80+
0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
308849,0,1,0,0,0,0,0,0,0,0,0,0,0
308850,0,0,0,0,0,0,0,0,0,1,0,0,0
308851,0,0,1,0,0,0,0,0,0,0,0,0,0
308852,0,0,0,0,0,0,0,0,0,1,0,0,0


# **Concatenate processed Numerical and Categorical columns**

In [92]:
df_obj = df_obj.drop(['Age_Category'], axis=1)
df_X = pd.concat([scaled_df_num,df_obj,df_age_cat], axis=1)

**Features (X)**

In [96]:
df_X.head()

,Height_(cm),Weight_(kg),BMI,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption,General_Health,Checkup,Exercise,...,Age_Category_35-39,Age_Category_40-44,Age_Category_45-49,Age_Category_50-54,Age_Category_55-59,Age_Category_60-64,Age_Category_65-69,Age_Category_70-74,Age_Category_75-79,Age_Category_80+
0,-1.934250,-2.386180,-2.159696,-0.621527,0.006625,0.059597,0.664502,0,3,0,...,0,0,0,0,0,0,0,1,0,0
1,-0.526857,-0.303547,-0.051548,-0.621527,0.006625,-1.012342,-0.267579,3,4,0,...,0,0,0,0,0,0,0,1,0,0
2,-0.714510,0.227770,0.742649,-0.133707,-0.716973,-0.811354,1.130543,3,4,1,...,0,0,0,0,0,1,0,0,0,0
3,0.880535,0.461569,0.015913,-0.621527,0.006625,0.997544,0.198462,0,4,1,...,0,0,0,0,0,0,0,0,1,0
4,1.912623,0.227770,-0.652562,-0.621527,-0.877772,-0.744358,-0.733620,2,4,0,...,0,0,0,0,0,0,0,0,0,1


In [97]:
df_X.columns

Index(['Height_(cm)', 'Weight_(kg)', 'BMI', 'Alcohol_Consumption',
       'Fruit_Consumption', 'Green_Vegetables_Consumption',
       'FriedPotato_Consumption', 'General_Health', 'Checkup', 'Exercise',
       'Skin_Cancer', 'Other_Cancer', 'Depression', 'Diabetes', 'Arthritis',
       'Sex', 'Smoking_History', 'Age_Category_18-24', 'Age_Category_25-29',
       'Age_Category_30-34', 'Age_Category_35-39', 'Age_Category_40-44',
       'Age_Category_45-49', 'Age_Category_50-54', 'Age_Category_55-59',
       'Age_Category_60-64', 'Age_Category_65-69', 'Age_Category_70-74',
       'Age_Category_75-79', 'Age_Category_80+'],
      dtype='object')

In [98]:
df_X.shape

(308854, 30)

**Label (y)**

In [99]:
df['Heart_Disease'] = df['Heart_Disease'].map({'No':0,'Yes':1})
df_y = df['Heart_Disease']
print(df['Heart_Disease'].value_counts())

0    283883
1     24971
Name: Heart_Disease, dtype: int64


In [100]:
df_y.shape

(308854,)

# **Classifiers**

In [101]:
gbc = GradientBoostingClassifier(learning_rate=0.1,max_depth=9,min_samples_leaf=60,min_samples_split=10,n_estimators=31,random_state=7)

lgbm = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=4984, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

lda = LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)


ridge = RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True, max_iter=None, random_state=6409,
                solver='auto', tol=0.001)

rf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, 
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=2313, verbose=0,
                       warm_start=False)

# **Cross validation (StratifiedKFold, SMOTE)**

In [102]:
def CrossValidation(X,y,classifier):
    accuracy = []

    cv = StratifiedKFold(n_splits=10, random_state=None, shuffle=False)
    cv.get_n_splits(X, y)

    for train, test in cv.split(X, y):
        X_train, X_test = X.iloc[train], X.iloc[test]
        y_train, y_test = y.iloc[train], y.iloc[test]

        sm = SMOTE()
        X_train, y_train = sm.fit_resample(X_train, y_train)

        classifier.fit(X_train, y_train)

        prediction = classifier.predict(X_test)
        score = metrics.accuracy_score(y_test, prediction)
        accuracy.append(score)
        
    return accuracy

In [103]:
gbc_accuracy = CrossValidation(df_X, df_y, gbc)
gbc_accuracy_mean = (np.array(gbc_accuracy).mean()) * 100

In [104]:
lgbm_accuracy = CrossValidation(df_X, df_y, lgbm)
lgbm_accuracy_mean = (np.array(lgbm_accuracy).mean()) * 100

In [105]:
lda_accuracy = CrossValidation(df_X, df_y, lda)
lda_accuracy_mean = (np.array(lda_accuracy).mean()) * 100

In [106]:
ridge_accuracy = CrossValidation(df_X, df_y, ridge)
ridge_accuracy_mean = (np.array(ridge_accuracy).mean()) * 100

In [ ]:
rf_accuracy = CrossValidation(df_X, df_y, rf)
rf_accuracy_mean = (np.array(rf_accuracy).mean()) * 100

# **LSTM**

In [110]:
def lstm(X,y):
    input = keras.layers.Input(shape=(X.shape[1], 1))

    conv1d = keras.layers.Conv1D(32*2, 7)(input)
    lstm = keras.layers.LSTM(units=384)(conv1d)

    dense_layer_2 = keras.layers.Dense(100, activation="relu")(lstm)

    output_layer = keras.layers.Dense(3, activation="softmax")(dense_layer_2)

    model = keras.Model(inputs=input, outputs=output_layer)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    sm = SMOTE()
    X_train, y_train = sm.fit_resample(X_train, y_train)

    history = model.fit(X_train, y_train, epochs=1, batch_size=128,
                         validation_data=(X_test, y_test), verbose=1)
    scores = model.evaluate(X_test, y_test, batch_size=128, verbose=1)

    return scores[1]*100

In [111]:
lstm_accuracy = lstm(df_X, df_y)

604/604 [==============================] - 59s 97ms/step - loss: 0.4588 - accuracy: 0.7552


In [112]:
print('Model Accuracy (%):')
print('Gradient Boosting Classifier : ',gbc_accuracy_mean)
print('LGBM Classifier : ',lgbm_accuracy_mean)
print('Linear Discriminant Analysis : ',lda_accuracy_mean)
print('Ridge Classifier : ',ridge_accuracy_mean)
print('Random Forest : ',rf_accuracy_mean)
print('LSTM : ',lstm_accuracy)

Model Accuracy (%):
Gradient Boosting Classifier :  87.5792434708823
LGBM Classifier :  91.68215386812972
Linear Discriminant Analysis :  76.64689045222322
Ridge Classifier :  76.66502207415718
Random Forest :  89.65562832486042
LSTM :  75.51609873771667
